In [1]:
# Elastic Search에 소음 데이터 10000 * 60 건(1분 데이터)를 저장하는 프로그램 개발
#  → 10분 데이터중 임의의 위치에서 10000 * 60 건(1분 데이터)를 발췌하여 저장
#     소음값 사이에 스페이스(' ') 하나의 스트링으로 만들어 저장
#  → requests 패키지 활용(/jmsohn/esvib/get_esvib_data.ipynb 참조)
  
# 입력값: 소음 데이터 파일명
# 수행후 결과: 
#  - ElasticSearch 인덱스에 1분 소음 데이터 저장됨
#    인덱스명 : es-raw (기 생성되어 있음, 없을경우, 손팀장에게 생성 요청)
#    Doc Id: '0' 으로 고정
#   인덱스의 맵핑을 확인하여, 데이터 저장할 것

In [2]:
# 1. 소음데이터 txt 파일 읽기
# 2. 랜덤한 위치에서 10000 * 60건 읽기 
# 3. 소음데이터 사이에 개행문자를 제거하기
# 4. index열 제거하기
# 5. 'es-raw' 인덱스에 스페이스(' ')를 넣어 하나의 String으로 저장하기 

In [3]:
import numpy as np

import requests
import json

In [4]:
def get_sample_noise(noise_file_name):
    
    # 1. 소음데이터 txt 파일 읽기
    # 파일을 읽어 리스트로 만듦
    lines = []
    with open(noise_file_name, "r") as noise_file:
        lines = noise_file.readlines()
    
    # 2. 랜덤한 위치에서 10000 * 60건 읽기   
    start_indices = []
    start_indices.append(np.random.randint(5400000) + 1) # 1은 파일의 앞쪽, 데이터가 아닌 부분을 제거하기 위한 오프셋
    
    # 시작 위치 인덱스 부터 600000개 데이터 추출
    for start_index in start_indices:
        
        bundle = lines[start_index:start_index + 600000]     
        
        bundle_data = []
        for line in bundle:
            # 3. 소음데이터 사이에 개행문자를 제거하기
            stripped = line.rstrip()
            # 4. index열 제거하기
            bundle_data.append(stripped.split(',')[1])
           
    return bundle_data


In [5]:
# 소음데이터 파일
noise_txt_file = '/src/data/NoiseRawData_10000/0/r_10min.txt'

noise_list = []
    
print("read file: %s" %noise_txt_file)
sample_noise_list = get_sample_noise(noise_txt_file)
noise_list = sample_noise_list
print("read complete.")


read file: /src/data/NoiseRawData_10000/0/r_10min.txt
read complete.


In [6]:
# test
#file = open('/src/hyebin/esnoise/r_10min.txt', 'w')
#file.write(' '.join(noise_list)) 
#file.close()
    
#print("save complete.")

In [7]:
# 5.'es-raw' 인덱스에 스페이스(' ')를 넣어 하나의 String으로 저장하기
string_data = ' '.join(noise_list)

headers = {'Content-Type': 'application/json; charset=utf-8'}

doc = {
        'type': 'text',
        'index': 'false',
        'text': string_data
}

response = requests.put(url = "http://192.168.10.204:9200/es-raw/_doc/0",
                        headers = headers,
                        data = json.dumps(doc))

print(response)


<Response [200]>
